# 📊 Trader Behavior vs Bitcoin Market Sentiment


This notebook analyzes the relationship between **Hyperliquid trader performance** and the **Bitcoin Fear & Greed Index**, using historical trading data and sentiment classification.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Load data
historical_df = pd.read_csv("historical_data.csv")
fear_greed_df = pd.read_csv("fear_greed_index.csv")

# Convert timestamps
historical_df['date'] = pd.to_datetime(historical_df['Timestamp IST'], format='%d-%m-%Y %H:%M').dt.date
fear_greed_df['date'] = pd.to_datetime(fear_greed_df['date']).dt.date

# Merge data
merged_df = pd.merge(historical_df, fear_greed_df[['date', 'classification']], on='date', how='left')


## 📈 Summary Statistics by Sentiment

In [ ]:

merged_df.groupby('classification').agg({
    'Closed PnL': ['mean', 'median'],
    'Size USD': 'mean',
    'Execution Price': 'mean'
}).round(2)


## 🔍 Average Profit by Sentiment

In [ ]:

plt.figure(figsize=(10, 6))
avg_pnl_by_sentiment = merged_df.groupby('classification')['Closed PnL'].mean().sort_values()
sns.barplot(x=avg_pnl_by_sentiment.index, y=avg_pnl_by_sentiment.values, palette="viridis")
plt.title("Average Closed PnL by Market Sentiment")
plt.ylabel("Average Closed PnL ($)")
plt.xlabel("Sentiment Classification")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 📦 PnL Distribution by Sentiment

In [ ]:

plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_df, x='classification', y='Closed PnL', showfliers=False, palette='Set2')
plt.title("Closed PnL Distribution by Sentiment")
plt.ylabel("Closed PnL ($)")
plt.xlabel("Sentiment Classification")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 💼 Top Traders and Their Performance by Sentiment

In [ ]:

top_traders = merged_df.groupby("Account")["Closed PnL"].sum().sort_values(ascending=False).head(10).index
top_trader_df = merged_df[merged_df["Account"].isin(top_traders)]

plt.figure(figsize=(14, 7))
sns.boxplot(data=top_trader_df, x="classification", y="Closed PnL", hue="Account", showfliers=False)
plt.title("Top 10 Traders' PnL Distribution by Sentiment")
plt.xlabel("Market Sentiment")
plt.ylabel("Closed PnL ($)")
plt.xticks(rotation=45)
plt.legend(loc="upper right", bbox_to_anchor=(1.15, 1))
plt.tight_layout()
plt.show()
